In [5]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# تثبيت العشوائية
tf.random.set_seed(42)
np.random.seed(42)

# تحميل البيانات
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# تطبيع البيانات
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# One-hot encoding
encoder = OneHotEncoder(sparse_output=False)
y_train_oh = encoder.fit_transform(y_train.reshape(-1, 1))
y_test_oh = encoder.transform(y_test.reshape(-1, 1))

# نموذج CNN
def create_cnn_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.25),

        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.25),

        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# دوال الأوزان
def flatten_weights(model):
    weights = []
    for layer in model.layers:
        for w in layer.get_weights():
            weights.extend(w.flatten())
    return np.array(weights)

def reshape_weights(model, flat_weights):
    idx = 0
    for layer in model.layers:
        layer_weights = layer.get_weights()
        if layer_weights:
            new_weights = []
            for w in layer_weights:
                size = np.prod(w.shape)
                new_w = flat_weights[idx:idx+size].reshape(w.shape)
                new_weights.append(new_w)
                idx += size
            layer.set_weights(new_weights)

# دالة التقييم
def fitness_function(flat_weights, model_template, X, y):
    temp_model = tf.keras.models.clone_model(model_template)
    temp_model.build(input_shape=(None, 28, 28, 1))
    reshape_weights(temp_model, flat_weights)
    pred = temp_model(tf.convert_to_tensor(X), training=False).numpy()
    acc = np.mean(np.argmax(pred, axis=1) == np.argmax(y, axis=1))
    return 1.0 - acc

# GWO المبسطة
def GWO(model_template, X, y, n_agents=5, max_iter=5):
    dim = len(flatten_weights(model_template))
    alpha_pos, beta_pos, delta_pos = np.zeros(dim), np.zeros(dim), np.zeros(dim)
    alpha_score, beta_score, delta_score = float('inf'), float('inf'), float('inf')
    wolves = np.random.uniform(-1, 1, (n_agents, dim))

    for t in range(max_iter):
        for i in range(n_agents):
            fitness = fitness_function(wolves[i], model_template, X, y)
            if fitness < alpha_score:
                alpha_score, alpha_pos = fitness, wolves[i].copy()
            elif fitness < beta_score:
                beta_score, beta_pos = fitness, wolves[i].copy()
            elif fitness < delta_score:
                delta_score, delta_pos = fitness, wolves[i].copy()

        a = 2 - t * (2 / max_iter)
        for i in range(n_agents):
            for j in range(dim):
                r1, r2 = np.random.rand(), np.random.rand()
                A1, C1 = 2*a*r1 - a, 2*r2
                D_alpha = abs(C1*alpha_pos[j] - wolves[i][j])
                X1 = alpha_pos[j] - A1*D_alpha

                r1, r2 = np.random.rand(), np.random.rand()
                A2, C2 = 2*a*r1 - a, 2*r2
                D_beta = abs(C2*beta_pos[j] - wolves[i][j])
                X2 = beta_pos[j] - A2*D_beta

                r1, r2 = np.random.rand(), np.random.rand()
                A3, C3 = 2*a*r1 - a, 2*r2
                D_delta = abs(C3*delta_pos[j] - wolves[i][j])
                X3 = delta_pos[j] - A3*D_delta

                wolves[i][j] = (X1 + X2 + X3) / 3.0

        print(f"Iteration {t+1}/{max_iter}, Best Accuracy: {1 - alpha_score:.4f}")

    return alpha_pos

# اختيار عينة صغيرة
X_sample, y_sample = shuffle(X_train, y_train_oh, random_state=42)
X_sample, y_sample = X_sample[:200], y_sample[:200]

# تطبيق GWO
model = create_cnn_model()
best_weights = GWO(model, X_sample, y_sample)
reshape_weights(model, best_weights)

# التدريب النهائي
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

model.fit(X_train, y_train_oh,
          epochs=30,
          batch_size=128,
          validation_split=0.1,
          callbacks=[early_stop, reduce_lr],
          verbose=2)

# التقييم النهائي
loss, acc = model.evaluate(X_test, y_test_oh, verbose=0)
print("Final Test Accuracy:", round(acc * 100, 2), "%")

Iteration 1/5, Best Accuracy: 0.1050
Iteration 2/5, Best Accuracy: 0.1050
Iteration 3/5, Best Accuracy: 0.1050
Iteration 4/5, Best Accuracy: 0.1050
Iteration 5/5, Best Accuracy: 0.1050
Epoch 1/30
422/422 - 333s - 788ms/step - accuracy: 0.2636 - loss: 7.1945 - val_accuracy: 0.0973 - val_loss: 22.0239 - learning_rate: 5.0000e-04
Epoch 2/30
422/422 - 280s - 664ms/step - accuracy: 0.4551 - loss: 1.5015 - val_accuracy: 0.5422 - val_loss: 1.2770 - learning_rate: 5.0000e-04
Epoch 3/30
422/422 - 265s - 628ms/step - accuracy: 0.5332 - loss: 1.2594 - val_accuracy: 0.5743 - val_loss: 1.0798 - learning_rate: 5.0000e-04
Epoch 4/30
422/422 - 251s - 594ms/step - accuracy: 0.5867 - loss: 1.1186 - val_accuracy: 0.6170 - val_loss: 0.9813 - learning_rate: 5.0000e-04
Epoch 5/30
422/422 - 255s - 604ms/step - accuracy: 0.6328 - loss: 1.0096 - val_accuracy: 0.6728 - val_loss: 0.8357 - learning_rate: 5.0000e-04
Epoch 6/30
422/422 - 231s - 547ms/step - accuracy: 0.6571 - loss: 0.9359 - val_accuracy: 0.6793 - v